## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import hour
from pyspark.sql.functions import mean
from pyspark.sql.functions import count

# File location and type
file_location = "/FileStore/tables/WeatherEvents_Jan2016_Dec2021.csv"




# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header = True, inferSchema = True)

#Create a dataframe of set number of rows from larger dataframe
df_50 = df.limit(50)

#Caching frequently used df for performance
df_50.cache()

In [0]:
#Prints schema of the data with inferred data types
df.printSchema()

root
 |-- EventId: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- StartTime(UTC): timestamp (nullable = true)
 |-- EndTime(UTC): timestamp (nullable = true)
 |-- Precipitation(in): double (nullable = true)
 |-- TimeZone: string (nullable = true)
 |-- AirportCode: string (nullable = true)
 |-- LocationLat: double (nullable = true)
 |-- LocationLng: double (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)



In [0]:
#Show data (default cuts off at 20 lines)
df.show()

+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId|Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|    W-1|Snow|   Light|2016-01-06 23:14:00|2016-01-07 00:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-2|Snow|   Light|2016-01-07 04:14:00|2016-01-07 04:54:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-3|Snow|   Light|2016-01-07 05:54:00|2016-01-07 15:34:00|             0.03|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-4|Snow|   Light|2016-

In [0]:
#Selecting a specific column
df.select("EventId").show()

+-------+
|EventId|
+-------+
|    W-1|
|    W-2|
|    W-3|
|    W-4|
|    W-5|
|    W-6|
|    W-7|
|    W-8|
|    W-9|
|   W-10|
|   W-11|
|   W-12|
|   W-13|
|   W-14|
|   W-15|
|   W-16|
|   W-17|
|   W-18|
|   W-19|
|   W-20|
+-------+
only showing top 20 rows



In [0]:
#Count the number of rows in dataframe
df.count()

Out[5]: 7479165

In [0]:
#Summary of basic stats for a dataframe
df_50.describe().show()

+-------+-------+-----+--------+------------------+-----------+-----------+-----------------+-------------------+--------+--------+-----+-------+
|summary|EventId| Type|Severity| Precipitation(in)|   TimeZone|AirportCode|      LocationLat|        LocationLng|    City|  County|State|ZipCode|
+-------+-------+-----+--------+------------------+-----------+-----------+-----------------+-------------------+--------+--------+-----+-------+
|  count|     50|   50|      50|                50|         50|         50|               50|                 50|      50|      50|   50|     50|
|   mean|   null| null|    null|            0.0036|       null|       null|38.09719999999997|-106.16889999999994|    null|    null| null|81149.0|
| stddev|   null| null|    null|0.0165072649436049|       null|       null|              0.0|                0.0|    null|    null| null|    0.0|
|    min|    W-1| Cold|   Light|               0.0|US/Mountain|       K04V|          38.0972|          -106.1689|Saguache|Sa

In [0]:
#Returns a df with only distinct rows
df_50.distinct().show()

+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId|Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|    W-1|Snow|   Light|2016-01-06 23:14:00|2016-01-07 00:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-2|Snow|   Light|2016-01-07 04:14:00|2016-01-07 04:54:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-3|Snow|   Light|2016-01-07 05:54:00|2016-01-07 15:34:00|             0.03|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-4|Snow|   Light|2016-

In [0]:
#Return a df based on a condition 

#df_50.filter(col("Precipitation(in)") > 0).show() or =>
df_50.filter((df['Precipitation(in)'] > 0) & (hour(df['StartTime(UTC)']) > 8 )).show()

+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId|Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|   W-31|Snow|   Light|2016-01-24 20:35:00|2016-01-24 21:35:00|             0.01|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+



In [0]:
#Perform mathematical functions based on reacurring groups 
df_100000 = df.limit(100000)
df_100000.groupBy("City").agg(mean("Precipitation(in)")).show()
#can replace mean with sum, count, max, ....

+------------+----------------------+
|        City|avg(Precipitation(in))|
+------------+----------------------+
|    Saguache|   0.01461400359066428|
|       Altus|    0.0459024505183789|
|  Albert Lea|  0.038678832116788464|
| Baton Rouge|   0.23466420664206533|
|      Sparta|   0.07840802178757109|
|      Driggs|   0.05034528552456856|
|Grand Rapids|   0.09549455244605781|
|  Hutchinson|   0.15098298676748526|
|    Kemmerer|   0.00323883161512027|
|  Pittsfield|   0.10000863110650714|
| Kansas City|   0.17656002875628996|
|       Tulsa|   0.21223979753242464|
|  Las Cruces|   0.04557059961315286|
|       Davis|   0.21208454206382027|
|     Roebuck|   0.08679707753947649|
|     Sanford|   0.06515279453156404|
|    Westerly|   0.12850299401197499|
|   Worcester|   0.10764958097836587|
|   Charlotte|   0.05319019414280974|
|Williamsburg|    0.4768749999999997|
+------------+----------------------+
only showing top 20 rows



In [0]:
#Same as above, count function 
df_100000.groupBy("State").agg(count("EventId")).show();

+-----+--------------+
|State|count(EventId)|
+-----+--------------+
|   CO|          2228|
|   OK|          7405|
|   MN|          8706|
|   LA|          4065|
|   WI|          7923|
|   ID|          4518|
|   MI|         10759|
|   KS|          2645|
|   WY|          3492|
|   MA|         10924|
|   MO|          2782|
|   NM|          1551|
|   NC|          7387|
|   SC|          4243|
|   RI|          4676|
|   VA|           400|
|   CT|          3567|
|   OR|          9346|
|   ND|          3383|
+-----+--------------+



In [0]:
#Sorting order of rows in a dataframe by column
df_50.orderBy("State", "Precipitation(in)", ascending = False).show()

+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId|Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|   W-44|Snow|   Light|2016-02-01 08:15:00|2016-02-01 12:35:00|             0.11|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-3|Snow|   Light|2016-01-07 05:54:00|2016-01-07 15:34:00|             0.03|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-43|Snow|   Light|2016-02-01 05:35:00|2016-02-01 07:35:00|             0.03|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-31|Snow|   Light|2016-

In [0]:
#Drop duplicate rows when specified "subset" of columns are the same 
df_50.dropDuplicates(subset = ["Precipitation(in)", "Type", "Severity", "ZipCode"]).show()

+-------+-----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId| Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+-----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|   W-11| Cold|  Severe|2016-01-09 16:54:00|2016-01-09 20:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-38|  Fog|Moderate|2016-01-31 15:35:00|2016-01-31 16:15:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-7|  Fog|  Severe|2016-01-09 12:54:00|2016-01-09 15:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|    W-1| Snow|   Ligh

In [0]:
#Select random sample of data
df_50.sample(fraction = 0.1).show()

+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|EventId|Type|Severity|     StartTime(UTC)|       EndTime(UTC)|Precipitation(in)|   TimeZone|AirportCode|LocationLat|LocationLng|    City|  County|State|ZipCode|
+-------+----+--------+-------------------+-------------------+-----------------+-----------+-----------+-----------+-----------+--------+--------+-----+-------+
|    W-8|Snow|   Light|2016-01-09 15:34:00|2016-01-09 16:14:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-11|Cold|  Severe|2016-01-09 16:54:00|2016-01-09 20:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-14|Snow|   Light|2016-01-10 10:14:00|2016-01-10 10:34:00|              0.0|US/Mountain|       K04V|    38.0972|  -106.1689|Saguache|Saguache|   CO|  81149|
|   W-18|Cold|  Severe|2016-